In [7]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import pickle
from collections import Counter
import json

print(tf.__version__)

gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

2.16.2
1 Physical GPUs, 1 Logical GPUs


In [32]:
# clean for debug purpose only 

trainData = None
valData = None
testData = None
model = None

tf.keras.backend.clear_session(free_memory=True)


## Loading the data

TODO

In [15]:
# load images and labels 


labelSet = Counter()
dataDict = {}
vocab = set()


datasetLen = 0
with open("./A2_train_v3.jsonl", "r") as jsonFile:
	for line in jsonFile:
		datasetLen += 1
		loadedLine = json.loads(line)
		if loadedLine["Image_ID"] not in dataDict:
			dataDict[loadedLine["Image_ID"]] = []

		labelSet[loadedLine["Label"]] += 1

		hypo = [''.join(char for char in word if char.isalnum()) for word in loadedLine["Hypothesis"].lower().split()]
		vocab.update(hypo)

		dataDict[loadedLine["Image_ID"]].append((hypo, loadedLine["Label"]))

labelTuple = tuple(labelSet.keys())
vocab = list(vocab)

vocabIndex = {vocab[i]: i for i in range(len(vocab))}

print(len(dataDict.keys()))
print(len(vocabIndex))
print(datasetLen)
print(labelSet)

19573
9274
39129
Counter({'entailment': 19619, 'contradiction': 19510})


## Creating the tensorflow dataset

TODO

In [16]:
# load the glove embeddings 

def getGlove():
  print('Downloading glove')
  !wget https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
  !unzip -q glove.6B.zip

def generateMatrix(dim):
    print('parsing glove data')
    embeddingMatrix = np.zeros((len(vocab), dim))
    embeddedVocab = []
    
    with open(f'glove.6B.{dim}d.txt', encoding="utf-8") as gloveFile:
      for line in gloveFile:
        values = line.split()
        word = values[0]
    
        if word in vocab:
          embeddingMatrix[vocabIndex[word]] = np.asarray(values[1:], dtype='float32')
          embeddedVocab.append(word)

    print(f'embedded {len(embeddedVocab)} out of {len(vocab)}')
    return embeddingMatrix


embeddingDim = 100
embeddingMatrix = None

if not os.path.isfile(f'glove.6B.{embeddingDim}d.txt'):
  getGlove()

if os.path.isfile(f'embeddingMatrix.{embeddingDim}d.pkl'):
  with open(f'embeddingMatrix.{embeddingDim}d.pkl', 'rb') as f:
    embeddingMatrix = pickle.load(f)

  print('loaded matrix')
else:
  embeddingMatrix = generateMatrix(embeddingDim)

  with open(f'embeddingMatrix.{embeddingDim}d.pkl', 'ab') as f:
    pickle.dump(embeddingMatrix, f)

  print('saved matrix')

loaded matrix


In [17]:
maxLen = 128

X1array = []
X2array = []
YArray = []

for key, hypoAndLabels in dataDict.items():
	img = f'./A2_Images/{key}.jpg'

	for hypo, label in hypoAndLabels:
		label = tf.convert_to_tensor([labelTuple.index(label)])
		label.set_shape([1])

		hypo = [vocabIndex[word] for word in hypo]
		hypo = tf.convert_to_tensor(tf.keras.preprocessing.sequence.pad_sequences([hypo], maxlen=maxLen)[0])
		hypo.set_shape([maxLen])

		X1array.append(img)
		X2array.append(hypo)
		YArray.append(label)

X1Numpy = np.array(X1array)
X2Numpy = np.array(X2array)
YNumpy = np.array(YArray, dtype='uint8')

dataset = tf.data.Dataset.from_tensor_slices(({'image': X1Numpy, 'text': X2Numpy}, YNumpy))

def getImage(path):
	img = tf.io.read_file(path)
	img = tf.io.decode_image(img, channels=3)
	img = tf.image.resize(img, (224, 224), method='nearest')
	return img

def getImageWrapper(x, y):
	img = tf.py_function(func=getImage, inp=[x['image']], Tout=tf.uint8)
	img.set_shape([224, 224, 3])

	x['image'] = img 
	return x, y

dataset = dataset.map(getImageWrapper, num_parallel_calls=tf.data.AUTOTUNE)

In [18]:
for i in dataset.take(1):
	print(i)

({'image': <tf.Tensor: shape=(224, 224, 3), dtype=uint8, numpy=
array([[[ 55, 118, 162],
        [ 53, 117, 165],
        [ 45, 113, 162],
        ...,
        [179, 177, 164],
        [175, 173, 161],
        [175, 173, 160]],

       [[ 61, 122, 169],
        [ 54, 118, 166],
        [ 53, 119, 169],
        ...,
        [180, 178, 165],
        [174, 173, 155],
        [176, 174, 159]],

       [[ 58, 122, 168],
        [ 57, 121, 169],
        [ 54, 120, 170],
        ...,
        [177, 175, 162],
        [179, 178, 158],
        [176, 174, 159]],

       ...,

       [[129, 132, 141],
        [149, 149, 157],
        [168, 168, 170],
        ...,
        [157, 130, 123],
        [158, 140, 130],
        [155, 140, 133]],

       [[  4,   5,  26],
        [ 15,  12,  31],
        [ 28,  24,  38],
        ...,
        [155, 128, 121],
        [158, 139, 132],
        [155, 140, 133]],

       [[  8,   7,  12],
        [ 12,   8,   7],
        [ 17,  12,   8],
        ...,
        [1

2025-10-07 02:32:50.860475: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [20]:
testSize = int(datasetLen * 0.05)
valSize = int(datasetLen * 0.05)
trainSize = int(datasetLen - testSize - valSize)
batchSize = 100

def optimize(ds):
	ds = ds.batch(batchSize) 
	ds = ds.cache() 
	ds = ds.prefetch(buffer_size=tf.data.AUTOTUNE)
	
	return ds

def getTest(ds):
	ds = ds.take(testSize) 
	ds = optimize(ds)
	
	return ds

def getVal(ds):
	ds = ds.skip(testSize)
	ds = ds.take(valSize) 
	ds = optimize(ds)

	return ds

def getTrain(ds):
	ds = ds.skip(valSize + testSize)
	ds = ds.take(trainSize)
	ds = optimize(ds)

	return ds

testDS = getTest(dataset)
valDS = getVal(dataset)
trainDS = getTrain(dataset)


print(f"test data batches {tf.data.experimental.cardinality(testDS).numpy()}")
print(f"val data batches {tf.data.experimental.cardinality(valDS).numpy()}")
print(f"train data batches {tf.data.experimental.cardinality(trainDS).numpy()}")
print(f'ratios test:{testSize} val:{valSize} train:{trainSize}')


test data batches 20
val data batches 20
train data batches 353
ratios test:1956 val:1956 train:35217


# Training

ToDo


In [21]:
# Compiling
inputImageLayer = tf.keras.layers.Input(shape=(224, 224, 3), name='image')

# imagenet pre made model
baseModel = tf.keras.applications.MobileNetV3Large(
  weights='imagenet',
  include_top=False,
  pooling='max'
)

baseModel.trainable = False
convoLayers = baseModel(inputImageLayer, training=False)
denseImage = tf.keras.layers.Dense(256, activation='relu', name='denseImage')(convoLayers)

inputTextLayer = tf.keras.layers.Input(shape=(maxLen,), name='text')
embeddingText = tf.keras.layers.Embedding(len(vocab), embeddingDim, mask_zero=True, weights=[embeddingMatrix], trainable=False, name='embedding')(inputTextLayer)
denseText = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(128, activation='relu'), name='denseText')(embeddingText)
attentionText = tf.keras.layers.MultiHeadAttention(8, maxLen, name='attentionText')(denseText, denseText, denseText)
denseText2 = tf.keras.layers.Dense(256, activation='relu', name='denseText2')(attentionText)

combinedLayer = tf.keras.layers.Dot((2, 1), normalize=True, name='combined')([denseText2, denseImage])
dense1 = tf.keras.layers.Dense(256, activation='relu')(combinedLayer)
dense2 = tf.keras.layers.Dense(256, activation='relu')(dense1)
output = tf.keras.layers.Dense(1, name='output', activation='sigmoid')(dense2)

model = tf.keras.Model(inputs=[inputImageLayer, inputTextLayer], outputs=output)

model.compile(
  optimizer='adam',
  loss='binary_crossentropy',
  metrics=['accuracy']
)
model.summary()


12683000/12683000 [==============================] - 2s 0us/step
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 text (InputLayer)           [(None, 128)]                0         []                            
                                                                                                  
 embedding (Embedding)       (None, 128, 100)             927400    ['text[0][0]']                
                                                                                                  
 denseText (TimeDistributed  (None, 128, 128)             12928     ['embedding[0][0]']           
 )                                                                                                
                                                                                                  
 image (InputLayer)          

In [22]:
# Training

history = model.fit(
  trainDS,
	validation_data=valDS,
  epochs=8,
  batch_size=batchSize
)

Epoch 1/8


2025-10-07 02:34:53.068683: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8907
I0000 00:00:1759804504.728324   13755 service.cc:145] XLA service 0x7f9b0dbba5d0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1759804504.728357   13755 service.cc:153]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
2025-10-07 02:35:05.021926: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1759804505.419347   13755 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


353/353 [==============================] - 170s 336ms/step - loss: 0.6207 - accuracy: 0.6296 - val_loss: 0.5578 - val_accuracy: 0.7188
Epoch 2/8
353/353 [==============================] - 44s 124ms/step - loss: 0.5208 - accuracy: 0.7332 - val_loss: 0.5241 - val_accuracy: 0.7265
Epoch 3/8
353/353 [==============================] - 44s 124ms/step - loss: 0.4960 - accuracy: 0.7519 - val_loss: 0.5151 - val_accuracy: 0.7306
Epoch 4/8
353/353 [==============================] - 44s 124ms/step - loss: 0.4796 - accuracy: 0.7621 - val_loss: 0.5112 - val_accuracy: 0.7393
Epoch 5/8
353/353 [==============================] - 44s 124ms/step - loss: 0.4636 - accuracy: 0.7724 - val_loss: 0.5054 - val_accuracy: 0.7403
Epoch 6/8
353/353 [==============================] - 44s 124ms/step - loss: 0.4450 - accuracy: 0.7832 - val_loss: 0.5012 - val_accuracy: 0.7413
Epoch 7/8
353/353 [==============================] - 44s 124ms/step - loss: 0.4284 - accuracy: 0.7930 - val_loss: 0.5007 - val_accuracy: 0.7444
E

In [43]:
model.save('./final.keras')